In [1]:
# coding: utf-8

from  __future__ import absolute_import
from __future__ import print_function
from ImageDataGeneratorCustom import ImageDataGeneratorCustom
import numpy as np
from keras.applications.vgg16 import VGG16
from keras.layers import *
from keras.models import Model, load_model
from keras.preprocessing.image import load_img, img_to_array
import tensorflow as tf
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
import tensorflow_hub as hub
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor

import tensorflow as tf
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from keras import layers
from keras import optimizers
import shutil
import random
from PIL import Image
import numpy as np
import logging
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from keras import layers
import tensorflow_hub as hub
import shutil
import random
from PIL import Image
import pickle
import shutil
from tensorflow import debugging
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import tensorflow as tf
from pathlib import Path
from keras import applications
from keras import layers
from keras import losses
from keras import optimizers
from keras import metrics
from keras import Model
from keras.applications import resnet
from keras.callbacks import ModelCheckpoint
import tensorflow_addons as tfa
from keras.regularizers import l2
from keras.activations import *


import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)


print(tf.__version__)

# tf.config.run_functions_eagerly(True)
# tf.data.experimental.enable_debug_mode()

model_path = "./deep_ranking"

# batch_size = 96
# batch_size = 8
batch_size = 24
# batch_size = 3
# batch_size = 1

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\experimental\enable_hist_gradient_boosting.py:16: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


2.8.0


In [2]:
features = {}
# load the features dictionary from the file
with open('features_merged.pickle', 'rb') as handle:
    features = pickle.load(handle)

feature_dim = features["02461"].shape[0]

# Model

a lot of the code comes from https://keras.io/examples/vision/siamese_network/
and https://github.com/akarshzingade/image-similarity-deep-ranking/blob/master/deepRanking.py

In [3]:


embedding = tf.keras.Sequential([
    # tf.keras.layers.Dense(512, activation='relu'),
    # tf.keras.layers.BatchNormalization(),
    # tf.keras.layers.Dense(256, activation='relu'),
    # tf.keras.layers.BatchNormalization(),
    # tf.keras.layers.Dense(256, activation='linear'),

    # GlobalAveragePooling2D(),

    # tf.keras.layers.Dense(1024, activation='linear'),


    tf.keras.layers.Dense(4096, activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)),
    tf.keras.layers.Dropout(0.6),
    tf.keras.layers.Dense(4096, activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)),
    tf.keras.layers.Dropout(0.6),

    # tf.keras.layers.Dense(8192, activation='relu', kernel_regularizer=l2(0.001)),
    # tf.keras.layers.Dropout(0.6),
    # tf.keras.layers.Dense(8192, activation='relu', kernel_regularizer=l2(0.001)),
    # tf.keras.layers.Dropout(0.6),
    # tf.keras.layers.Dense(8192, activation='relu', kernel_regularizer=l2(0.001)),
    # tf.keras.layers.Dropout(0.6),
    # tf.keras.layers.Dense(8192, activation='relu', kernel_regularizer=l2(0.001)),
    # tf.keras.layers.Dropout(0.6),
    tf.keras.layers.Lambda(lambda  x: K.l2_normalize(x,axis=1))
])

class DistanceLayer(layers.Layer):
    """
    This layer is responsible for computing the distance between the anchor
    embedding and the positive embedding, and the anchor embedding and the
    negative embedding.
    """

    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def call(self, anchor, positive, negative):
        ap_distance = tf.reduce_sum(tf.square(anchor - positive), -1)
        an_distance = tf.reduce_sum(tf.square(anchor - negative), -1)
        return (ap_distance, an_distance)


anchor_input = layers.Input(name="anchor", shape=(feature_dim))
positive_input = layers.Input(name="positive", shape=(feature_dim))
negative_input = layers.Input(name="negative", shape=(feature_dim))

distances = DistanceLayer()(
    embedding(anchor_input),
    embedding(positive_input),
    embedding(negative_input),
)

model = Model(
    inputs=[anchor_input, positive_input, negative_input], outputs=distances
)

# model.summary()

In [4]:
# imgs = 2
# pred = model.predict((np.random.rand(imgs,1280),np.random.rand(imgs,1280),np.random.rand(imgs,1280)))
# print(pred)

In [5]:
#convert the features dictionary to a tensorflow hash table
keys = tf.constant([key for key in features.keys()])
values = tf.constant([val for val in features.values()])

table = tf.lookup.experimental.DenseHashTable(
    value_dtype=tf.float32,
    key_dtype=tf.string,
    empty_key="empty_key",
    deleted_key="deleted_key",
    default_value=[-1]*feature_dim,
    )

table.insert(keys, values)

In [6]:
#load strings from train_triplets.txt
train_triplets = np.loadtxt('./train_triplets.txt', dtype=str,delimiter = ' ')


dataset = tf.data.Dataset.from_tensor_slices(train_triplets)


@tf.function
def load_image(inputs):
    filenames = inputs
    # print(filenames)


    # print(table[filenames[0]])
    anchor = table[filenames[0]]
    positive = table[filenames[1]]
    negative = table[filenames[2]]

    output = (anchor,positive,negative)
    
    return output

dataset = dataset.map(load_image).cache().shuffle(buffer_size=1000).batch(batch_size).prefetch(20)

In [7]:
# a,p,n = np.loadtxt('./train_triplets.txt', dtype=str,delimiter = ' ',unpack=True)
# print(a.shape)

# train_triplets = ([features[s] for s in a], [features[s] for s in p], [features[s] for s in n])
# print(len(train_triplets))
# print(len(train_triplets[0]))
# print(len(train_triplets[0][0]))

In [8]:
# dataset = tf.data.Dataset.from_tensor_slices(train_triplets)

# dataset = dataset.cache().batch(batch_size).prefetch(300*batch_size)#.shuffle(300*batch_size)


In [9]:
# next(iter(dataset))

In [10]:
# # This function will plot images in the form of a grid with 1 row and 5 columns where images are placed in each column.
# def plotImages(images_arr):
#     fig, axes = plt.subplots(1, 5, figsize=(20,20))
#     axes = axes.flatten()
#     for img, ax in zip(images_arr, axes):
#         ax.imshow(img)
#     plt.tight_layout()
#     plt.show()

# augmented_images = [my_gen[0][0][0] for i in range(5)]
# plotImages(augmented_images)

In [11]:
class SiameseModel(Model):
    """The Siamese Network model with a custom training and testing loops.

    Computes the triplet loss using the three embeddings produced by the
    Siamese Network.

    The triplet loss is defined as:
       L(A, P, N) = max(‖f(A) - f(P)‖² - ‖f(A) - f(N)‖² + margin, 0)
    """

    def __init__(self, siamese_network, margin=0.5):
        super(SiameseModel, self).__init__()
        self.siamese_network = siamese_network
        self.margin = margin
        self.loss_tracker = metrics.Mean(name="loss")

    def call(self, inputs):
        return self.siamese_network(inputs)

    def train_step(self, data):
        # GradientTape is a context manager that records every operation that
        # you do inside. We are using it here to compute the loss so we can get
        # the gradients and apply them using the optimizer specified in
        # `compile()`.
        with tf.GradientTape() as tape:
            loss = self._compute_loss(data)

        # Storing the gradients of the loss function with respect to the
        # weights/parameters.
        gradients = tape.gradient(loss, self.siamese_network.trainable_weights)

        # Applying the gradients on the model using the specified optimizer
        self.optimizer.apply_gradients(
            zip(gradients, self.siamese_network.trainable_weights)
        )

        # Let's update and return the training loss metric.
        self.loss_tracker.update_state(loss)
        return {"loss": self.loss_tracker.result()}

    def test_step(self, data):
        loss = self._compute_loss(data)

        # Let's update and return the loss metric.
        self.loss_tracker.update_state(loss)
        return {"loss": self.loss_tracker.result()}

    def _compute_loss(self, data):
        # The output of the network is a tuple containing the distances
        # between the anchor and the positive example, and the anchor and
        # the negative example.
        ap_distance, an_distance = self.siamese_network(data)

        # Computing the Triplet Loss by subtracting both distances and
        # making sure we don't get a negative value.
        loss = ap_distance - an_distance
        loss = tf.maximum(loss + self.margin, 0.0)
        return loss

    @property
    def metrics(self):
        # We need to list our metrics here so the `reset_states()` can be
        # called automatically.
        return [self.loss_tracker]

In [12]:
siamese_model = SiameseModel(model,margin=0.5)
# siamese_model.compile(optimizer=tf.keras.optimizers.Adam(0.0001))
# siamese_model.compile(optimizer=tf.keras.optimizers.Adam(0.0000005))
siamese_model.compile(optimizer=tf.keras.optimizers.Adam(0.000001))


# class MyExponentialDecay(tf.keras.optimizers.schedules.ExponentialDecay):
#   def __call__(self, step):
#     return 1e-2 * super().__call__(step)

# initial_learning_rate = 1e-2
# wd = MyExponentialDecay(
#     initial_learning_rate,
#     decay_steps=14,
#     decay_rate=0.8,
#     staircase=True)


# step = tf.Variable(0, trainable=False)
# schedule = tf.optimizers.schedules.PiecewiseConstantDecay([10000, 15000], [1e-0, 1e-1, 1e-2])
# lr = 1e-1 * schedule(step)
# # wd = lambda: 1e-4 * schedule(step)
# opt = tfa.optimizers.AdamW(learning_rate=lr, weight_decay=wd)

# siamese_model.compile(optimizer=opt)


In [13]:

#this is needed, otherwise nothing works
pred = siamese_model.predict(next(iter(dataset)))
print(pred)

(array([0.516847  , 0.43181312, 0.4940071 , 0.5842757 , 0.5482377 ,
       0.43574452, 0.55073047, 0.42874548, 0.46582067, 0.4362592 ,
       0.38197672, 0.36572403, 0.48934537, 0.6045705 , 0.5729515 ,
       0.42375416, 0.36645746, 0.45063758, 0.4059661 , 0.46973476,
       0.6046382 , 0.5449496 , 0.45889103, 0.5293103 ], dtype=float32), array([0.45667267, 0.55163443, 0.46766412, 0.5917454 , 0.6091945 ,
       0.6577747 , 0.3886032 , 0.5379775 , 0.6182839 , 0.583356  ,
       0.50954795, 0.5639539 , 0.39135927, 0.49001554, 0.4421147 ,
       0.35350835, 0.40926147, 0.38413885, 0.5227969 , 0.48677132,
       0.43193632, 0.55536914, 0.42086077, 0.55242157], dtype=float32))


In [14]:
# print(my_gen.__getitem__(5)[0])

In [15]:
train_steps_per_epoch = int((59520)/batch_size)
# train_steps_per_epoch = int((59520*5)/batch_size)

# checkpoint_filepath = './checkpoint'
checkpoint_filepath = './checkpoints/checkpoint-{epoch}/'
# checkpoint_filepath = "saved-model-{epoch}.hdf5"
# checkpoint_filepath = "saved-model-{epoch}"
#save checkpoint after every epoch
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    # save_weights_only=True,
)


siamese_model.fit(dataset, epochs=100,steps_per_epoch=train_steps_per_epoch,batch_size=batch_size,callbacks=[model_checkpoint_callback])
# siamese_model.fit(train_triplets, epochs=2,steps_per_epoch=train_steps_per_epoch,batch_size=batch_size,callbacks=[model_checkpoint_callback])


Epoch 1/100
2480/2480 [==============================] - 49s 20ms/step - loss: 0.3320
Epoch 2/100
2480/2480 [==============================] - 43s 17ms/step - loss: 0.2782
Epoch 3/100
2480/2480 [==============================] - 43s 17ms/step - loss: 0.2635
Epoch 4/100
2480/2480 [==============================] - 43s 17ms/step - loss: 0.2526
Epoch 5/100
2480/2480 [==============================] - 43s 17ms/step - loss: 0.2433
Epoch 6/100
2480/2480 [==============================] - 43s 17ms/step - loss: 0.2350
Epoch 7/100
2480/2480 [==============================] - 45s 18ms/step - loss: 0.2273
Epoch 8/100
2480/2480 [==============================] - 47s 19ms/step - loss: 0.2201
Epoch 9/100
2480/2480 [==============================] - 48s 19ms/step - loss: 0.2131
Epoch 10/100
2480/2480 [==============================] - 47s 19ms/step - loss: 0.2064
Epoch 11/100
2480/2480 [==============================] - 47s 19ms/step - loss: 0.1998
Epoch 12/100
2480/2480 [============================

In [ ]:
# tf.keras.utils.plot_model(
#     model,
#     to_file="model.png",
#     show_shapes=True,
#     expand_nested=True,
    
# )